In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load dataset

In [ ]:
from shapely import set_precision
objects = pd.read_csv('/content/drive/MyDrive/dataset/madison/input/objects_update.csv')
# objects = gpd.GeoDataFrame(objects, geometry=gpd.GeoSeries.from_wkt(objects['geometry']), crs=4326)
# objects = objects.loc[objects['geometry'].is_valid].reset_index(drop=True)
# objects['geometry'] = objects['geometry'].apply(lambda s: set_precision(s, 1e-6))
objects.head()

,objectid,geometry,source,geom_type
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString


In [ ]:
objects.shape[0], objects.drop_duplicates(subset='objectid').shape[0]

(5776, 5776)

# LLM embeddings

In [ ]:
data = objects.copy().reset_index(drop=True)
data['geometry'] = data['geometry'].astype(str)
# data_rel = relations_small.copy().reset_index(drop=True)
data.head()

,objectid,geometry,source,geom_type
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString


In [ ]:
!pip install typing_extensions==4.5.0 --upgrade
!pip install tiktoken
!pip install openai --upgrade
!pip install --upgrade transformers
!pip install torch


  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.10.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-

In [ ]:
import tiktoken, openai

from google.colab import userdata

KEY = userdata.get('OPENAI_QSR')
ORG = userdata.get('OPENAI_ORG')
openai.organization = ORG
openai.api_key = KEY
# openai.Model.list()

### Geometry only

#### Sentence embeddings

### Geometry only

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/dataset/madison/output/objects_embeddings.csv')

In [ ]:
# import tiktoken



# embedding model parameters
embedding_model = "text-embedding-3-large" #"text-embedding-ada-002"
model_abbr = "embedding_large"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(embedding_encoding)
# encoding = tiktoken.encoding_for_model(embedding_model)

# omit reviews that are too long to embed
data["n_tokens"] = data['geometry'].astype(str).apply(lambda x: len(encoding.encode(x))) #_ada

data["n_tokens"].describe() #_ada

count    5776.000000
mean       42.521814
std        39.867749
min        10.000000
25%        12.000000
50%        24.000000
75%        58.000000
max       680.000000
Name: n_tokens, dtype: float64

In [ ]:
data["n_tokens"].sum()

245606

In [ ]:
client = openai.Client(api_key=KEY, organization=ORG)

def get_embedding(text, model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
data[model_abbr] = data['geometry'].apply(lambda x: get_embedding(x, model=embedding_model))

In [ ]:
data.to_csv('/content/drive/MyDrive/dataset/madison/input/objects_update_%s.csv'%model_abbr, index=False)

In [ ]:
data_large = pd.read_csv('/content/drive/MyDrive/dataset/madison/input/objects_update_embedding_large.csv')
# data_large = pd.merge(data_large, data[['objectid', model_abbr]], on='objectid', how='inner')
data_large.head()

,objectid,geometry,source,geom_type,n_tokens,embedding_large
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21,"[0.01921754889190197, 0.01929466612637043, -0...."
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22,"[0.021175693720579147, 0.015708766877651215, -..."
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24,"[0.011615517549216747, 0.02089725434780121, -0..."
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24,"[0.0076748463325202465, 0.030930323526263237, ..."
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23,"[0.03258766606450081, 0.024168409407138824, -0..."


In [ ]:
data_ada = pd.read_csv('/content/drive/MyDrive/dataset/madison/input/objects_update_embeddings.csv')
data= pd.merge(data_ada, data_large[['objectid', 'embedding_large']], on='objectid', how='inner')
data.head()

,objectid,geometry,source,geom_type,n_tokens,embedding_ada,embedding_large
0,"road_(6231530142, 3861040918)","LINESTRING (-89.4719 43.05, -89.472 43.05)",link,LineString,21,"[-0.014463399536907673, -0.004633771721273661,...","[0.01921754889190197, 0.01929466612637043, -0...."
1,"road_(53591318, 1179412798)","LINESTRING (-89.477 43.0693, -89.477 43.0681)",link,LineString,22,"[-0.014376881532371044, -0.004993679467588663,...","[0.021175693720579147, 0.015708766877651215, -..."
2,"road_(1178722373, 1178722211)","LINESTRING (-89.4626 43.0553, -89.4627 43.0554)",link,LineString,24,"[-0.015243860892951488, -0.004274553153663874,...","[0.011615517549216747, 0.02089725434780121, -0..."
3,"road_(53430628, 53468282)","LINESTRING (-89.3045 43.0246, -89.3053 43.0252)",link,LineString,24,"[-0.016633406281471252, -0.0014020365197211504...","[0.0076748463325202465, 0.030930323526263237, ..."
4,"road_(3444117858, 6680557104)","LINESTRING (-89.5081 43.0066, -89.5072 43.006)",link,LineString,23,"[-0.010978729464113712, -0.005506434012204409,...","[0.03258766606450081, 0.024168409407138824, -0..."


In [ ]:
  data.to_csv('/content/drive/MyDrive/dataset/madison/input/objects_embeddings.csv', index=False)